## Imports

In [1]:
import copy
import numpy as np
import pandas as pd
import multiprocessing
import matplotlib.pyplot as plt

import defs

from sklearn.svm import SVC
from collections import Counter
from sklearn.manifold import TSNE
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

## Loading Dataset

In [2]:
n_features = 16

In [3]:
dataset = pd.read_csv("Resources/pendigits", header=None)

In [4]:
dataset_test = pd.read_csv("Resources/pendigits.t", header=None)

In [5]:
dataset.head()

,0
0,8 1:47 2:100 3:27 4:81 5:57 6:37 7:26 10:23 11...
1,2 2:89 3:27 4:100 5:42 6:75 7:29 8:45 9:15 10:...
2,1 2:57 3:31 4:68 5:72 6:90 7:100 8:100 9:76 10...
3,4 2:100 3:7 4:92 5:5 6:68 7:19 8:45 9:86 10:34...
4,1 2:67 3:49 4:83 5:100 6:100 7:81 8:80 9:60 10...


In [6]:
dataset.shape, dataset_test.shape

((7494, 1), (3498, 1))

In [7]:
def get_dataset(dataset):
    X = []
    y = []

    for idx, i in enumerate(dataset[0]):
        k = [0] * n_features
        p = i.strip().split(" ")
        y.append(int(p[0]))
        for j in p[1:]:
            k[int(j.split(":")[0])-1] = float(j.split(":")[1])
        X.append(k)
    return np.array(X), np.array(y)

In [8]:
X_train_raw, y_train = get_dataset(dataset)

In [9]:
X_test_raw, y_test = get_dataset(dataset_test)

In [10]:
n_classes = np.unique(y_train).shape[0]

In [11]:
X_train_raw.shape, y_train.shape

((7494, 16), (7494,))

In [12]:
X_test_raw.shape, y_test.shape

((3498, 16), (3498,))

In [13]:
from sklearn.decomposition import PCA

In [14]:
pca = PCA(n_components=2)

In [15]:
X_train = pca.fit_transform(X_train_raw)

In [16]:
X_test = pca.transform(X_test_raw)

In [17]:
pca.explained_variance_ratio_

array([0.28506749, 0.24400191])

### Required Functions

In [18]:
def plot_tsne(X, y):
    tsne = TSNE(n_components=2, random_state=0)
    k = tsne.fit_transform(X)
    plt.scatter(k[:, 0], k[:, 1], c=y)
    plt.show()

In [19]:
def linear_kernel(x, y, c=2):
    return np.dot(x, y.T) + c

In [20]:
def rbf_kernel(X, y, sigma = 3):
    K = np.zeros((X.shape[0], y.shape[0]))
    for i, x_ in enumerate(X):
        for j, y_ in enumerate(y):
            K[i, j] = np.linalg.norm(x_-y_)**2
    return np.exp(-(1.0/2.0*(sigma**2))*K)

### Analysis

In [21]:
# plot_tsne(X_train, y_train)

In [22]:
# plot_tsne(X_test, y_test)

# Baselines

In [23]:
svm = SVC(kernel='linear')
svm.fit(X_train, y_train)
print(svm.score(X_train, y_train))
print(svm.score(X_test, y_test))

0.6323725647184414
0.6092052601486564


In [32]:
svm = SVC(kernel='linear')
svm.fit(X_train_raw, y_train)
print(svm.score(X_train_raw, y_train))
print(svm.score(X_test_raw, y_test))

0.9930611155591139
0.9505431675242996


In [24]:
svm = SVC(kernel='rbf')
svm.fit(X_train, y_train)
print(svm.score(X_train, y_train))
print(svm.score(X_test, y_test))

/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.8820389645049372
0.6297884505431676


In [33]:
svm = SVC(kernel='rbf')
svm.fit(X_train_raw, y_train)
print(svm.score(X_train_raw, y_train))
print(svm.score(X_test_raw, y_test))

/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


1.0
0.13693539165237278


# Models
- OVR - Linear and RBF
- AVA - Linear and RBF

# OVO

In [25]:
def ovo_predcit_proba(X_train, X_test, svm_all):
    y_prob_train, y_prob_test = [], []

    for clf in svm_all:
        y_prob_train.append(clf.predict(X_train).tolist())
        y_prob_test.append(clf.predict(X_test).tolist())
    return np.array(y_prob_train).T, np.array(y_prob_test).T

In [26]:
def ovo_multiclass_convert(y, combinations):
    y_predicted = []

    for p in y:
        val = [0] * n_classes
        for i in range(0, len(p)):
            if p[i] <= 0:
                val[combinations[i][0]]= val[combinations[i][0]]+1
            else:
                val[combinations[i][1]]= val[combinations[i][1]]+1
        y_predicted.append(val.index(max(val)))
    
    return np.array(y_predicted)

In [27]:
def ovo_main(X_train, y_train, X_test, y_test, kernel):
    combinations = []
    for i in np.unique(y_train):
        for j in np.unique(y_train):
            if i < j:
                combinations.append([i,j])

    grid = []
    for i, j in combinations:
        grid.append((X_train, y_train, kernel, i, j))

    pool = multiprocessing.Pool()
    svm_all = pool.starmap(defs.ovo_svm_multipro, grid)

    y_proba_train, y_proba_test = ovo_predcit_proba(X_train, X_test, svm_all)

    y_predicted_train = ovo_multiclass_convert(y_proba_train, combinations)

    y_predicted_test = ovo_multiclass_convert(y_proba_test, combinations)

    print(kernel, "Train", accuracy_score(y_predicted_train, y_train))
    print(kernel, "Test", accuracy_score(y_predicted_test, y_test))

In [28]:
ovo_main(X_train, y_train, X_test, y_test, 'linear')

linear Train 0.6323725647184414
linear Test 0.6092052601486564


In [35]:
ovo_main(X_train_raw, y_train, X_test_raw, y_test, 'linear')

linear Train 0.9930611155591139
linear Test 0.9505431675242996


In [31]:
ovo_main(X_train, y_train, X_test, y_test, 'rbf')

rbf Train 0.8820389645049372
rbf Test 0.6297884505431676


In [30]:
ovo_main(X_train_raw, y_train, X_test_raw, y_test, 'rbf')

rbf Train 1.0
rbf Test 0.13693539165237278


In [36]:
ovo_main(X_train, y_train, X_test, y_test, linear_kernel)

<function linear_kernel at 0x7f21563f6bf8> Train 0.6323725647184414
<function linear_kernel at 0x7f21563f6bf8> Test 0.6092052601486564


In [ ]:
ovo_main(X_train, y_train, X_test, y_test, rbf_kernel)